<a href="https://colab.research.google.com/github/Collin-Campbell/DS-Unit-4-Sprint-1-NLP/blob/main/module3-document-classification/LS_DS_413_Document_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [363]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.3MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=c8ba46be3344089a9fdfd911ea69802b9c22e6f223d8ad7ceb8018dffb2a915f
  Stored in directory: /tmp/pip-ephem-wheel-cache-7pwccmcl/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [349]:
# https://www.kaggle.com/c/whiskey-reviews-ds20/overview

import pandas as pd

# You may need to change the path
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [350]:
import re
import string

def wrangle(text):
  stop_words = ['a','about','above','after','again','against','ain','all','along','am',
                  'an','and','any','are','aren',"aren't",'as','at','be','because',
                  'been','before','being','below','between','both','but','by','can',
                  'couldn',"couldn't",'d','did','didn',"didn't",'do','does','doesn','doesnt'
                  "doesn't",'doing','don',"don't",'down','during','each','every','few',
                  'for','from','full','further','had','hadn',"hadn't",'has','hasn',
                  "hasn't",'have','haven',"haven't",'having','he','her','here',
                  'hers','herself','him','himself','his','how','i','if','in','like',
                  'into','is','isn',"isn't",'it',"it's","it’s",'its','itself','just',
                  'll','m','ma','may','me','mightn',"mightn't",'more','most','mustn',
                  "mustn't",'my','myself','needn',"needn't",'no','nose','note','notes','nor','not',
                  'now','o','of','off','old','on','once','only','or','other','our',
                  'ours','ourselves','out','over','own','palate','re','s','same','shan',
                  "shan't",'she',"she's",'should',"should've",'shouldn',"shouldn't",
                  'so','some','sometimes','such','t','than','that',"that'll",'the','their',
                  'theirs','them','themselves','then','there','these','they',
                  'this','those','through','to','too','under','until','up','ve',
                  'very','was','wasn',"wasn't",'we','were','weren',"weren't",
                  'what','when','where','which','while','who','whom','why','will','—',
                  'with','won',"won't",'wouldn',"wouldn't",'y','year','yet','you',"you'd",
                  "you'll","you're","you've",'your','yours','yourself','yourselves','']

  # make all text lower case
  text = text.lower()

  # remove '\n'
  text = text.replace('\n', '')

  # remove punctuation
  text = ' '.join(word.strip(string.punctuation) for word in text.split())

  # remove '-'
  text = text.replace('-', '')

  cleaned_text = []
  for item in text.split():
    z = re.sub('[^a-z]', '', item)
    cleaned_text.append(z)

  # remove stopwords
  super_cleaned_text = ''
  for word in cleaned_text:
    if word not in stop_words:
      super_cleaned_text += (' ' + word)
    else:
      pass

  return super_cleaned_text.strip()

In [351]:
train['description'] = train['description'].apply(wrangle)

In [352]:
train.head()

,id,description,ratingCategory
0,1321,whisky batched leftover barrels returned wareh...,1
1,3861,uncommon exclusive bottling cask strength malt...,0
2,655,release port version amruts intermediate sherr...,1
3,555,single cask aged sherry butt interacted magnif...,1
4,1965,quite herbal aromas dried tarragon parsley dil...,1


In [353]:
train['description'][15]

'fragrant spicy honeyed vanilla peaches cream butterscotch cinnamon licorice red black light nuttiness toasted oak finish nice mouthfeel wellbalanced quite rich elegant whisky'

In [354]:
test['description'] = test['description'].apply(wrangle)

In [355]:
test.head()

,id,description
0,3461,style speyside single malt scotch color walnut...
1,2604,bright lively nice balance flavors zesty fruit...
2,3341,new olorosoforward chivas positioned split old...
3,3764,aged bourbon casks enhanced rioja wine casks m...
4,2306,freshness wood laced caramel delicate minty po...


### Define Pipeline Components

In [356]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

# Creating pipeline components
vect = TfidfVectorizer(ngram_range=(1,2))
rfc = RandomForestClassifier()
svd = TruncatedSVD()


# Defining the pipeline:

# LSA part of pipeline
lsa = Pipeline([
                # Vectorizer (i.e. data transformation)
                ('vect', vect), 
                # TruncatedSVD
                ('svd', svd)])

# Combining into one pipeline
pipe = Pipeline([
                 #LSA portion
                 ('lsa', lsa), 
                 # Classifier (in this case, random forest classifier)
                 ('clf', rfc)
                 ])

In [357]:
# split target and feature matrix

X = train['description']
y = train['ratingCategory']

print(X.shape)
print(y.shape)

assert len(X) == len(y)

(4087,)
(4087,)


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [358]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'lsa__svd__n_components': [10, 100, 250],
    'lsa__vect__max_df': (0.7,  0.8),
    'lsa__vect__min_df': (0.1, 0.2),
    'lsa__vect__max_features': (500, 1000),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators':[10, 100, 250]
}

grid_search = GridSearchCV(pipe,
                           parameters, 
                           cv=4, 
                           n_jobs=-1, 
                           verbose=1)



grid_search.fit(X, y)

Fitting 4 folds for each of 288 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed: 10.6min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsa',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [359]:
grid_search.best_params_

{'clf__max_depth': 5,
 'clf__n_estimators': 250,
 'lsa__svd__n_components': 10,
 'lsa__vect__max_df': 0.7,
 'lsa__vect__max_features': 1000,
 'lsa__vect__min_df': 0.1}

In [361]:
grid_search.best_score_

0.7085878546607352

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [270]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [271]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [272]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [274]:
subNumber = 5

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
# Completed above!!!!!!!!
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [364]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Function to return the vector for each sentence in a document
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [ ]:
X_train = get_word_vectors(X)

In [ ]:
classifier = RandomForestClassifier()

classifier.fit(X_train,y)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
      *   Sentiment Analysis is when one uses text analysis techniques in order to analyze the emotions being conveyed within text data.

    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
      *   Sentiment Analysis is a type of Document Classification.  For example, we could classify a document by its sentiment or by other classes such as subject type, etc.

    - How do create labeled sentiment data? Are those labels really sentiment?
      *   Define clear rules for each label.  The labels are, of course, subjective.

    - What are common applications of sentiment analysis?
      *   Customer reviews, restaurant reviews, consumer reviews...

2. Research why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?
      *   Word embeddings require large amounts of text data, and Neural Networks allow for more efficiency in this aspect. 